In [ ]:
#Esto es un cuaderno, para realizar el proyecto paso a paso

In [1]:
import pandas as pd

In [2]:
df_init=pd.read_csv("commercial_activity_df.csv")
print(df_init.head())

   Unnamed: 0   pk_cid pk_partition  entry_date entry_channel  \
0           0  1375586   2018-01-28  2018-01-12           KHL   
1           1  1050611   2018-01-28  2015-08-10           KHE   
2           2  1050612   2018-01-28  2015-08-10           KHE   
3           3  1050613   2018-01-28  2015-08-10           KHD   
4           4  1050614   2018-01-28  2015-08-10           KHE   

   active_customer             segment  
0              1.0   02 - PARTICULARES  
1              0.0  03 - UNIVERSITARIO  
2              0.0  03 - UNIVERSITARIO  
3              0.0  03 - UNIVERSITARIO  
4              1.0  03 - UNIVERSITARIO  


In [3]:
df_init.describe()

,Unnamed: 0,pk_cid,active_customer
count,5.962924e+06,5.962924e+06,5.962924e+06
mean,7.155425e+06,1.234930e+06,4.033845e-01
std,3.920784e+06,1.623020e+05,4.905767e-01
min,0.000000e+00,1.589100e+04,0.000000e+00
25%,4.240386e+06,1.112532e+06,0.000000e+00
50%,7.499346e+06,1.231097e+06,0.000000e+00
75%,1.020142e+07,1.352339e+06,1.000000e+00
max,1.364731e+07,1.553689e+06,1.000000e+00


In [5]:
df_init.set_index('Unnamed: 0', inplace=True)

In [6]:
df_init.describe()

,pk_cid,active_customer
count,5.962924e+06,5.962924e+06
mean,1.234930e+06,4.033845e-01
std,1.623020e+05,4.905767e-01
min,1.589100e+04,0.000000e+00
25%,1.112532e+06,0.000000e+00
50%,1.231097e+06,0.000000e+00
75%,1.352339e+06,1.000000e+00
max,1.553689e+06,1.000000e+00


In [7]:
print(df_init.head())

             pk_cid pk_partition  entry_date entry_channel  active_customer  \
Unnamed: 0                                                                    
0           1375586   2018-01-28  2018-01-12           KHL              1.0   
1           1050611   2018-01-28  2015-08-10           KHE              0.0   
2           1050612   2018-01-28  2015-08-10           KHE              0.0   
3           1050613   2018-01-28  2015-08-10           KHD              0.0   
4           1050614   2018-01-28  2015-08-10           KHE              1.0   

                       segment  
Unnamed: 0                      
0            02 - PARTICULARES  
1           03 - UNIVERSITARIO  
2           03 - UNIVERSITARIO  
3           03 - UNIVERSITARIO  
4           03 - UNIVERSITARIO  
